# Position-switched Data Reduction
----------------------------------

This notebook shows how to use `dysh` to calibrate an OnOff observation.
It retrieves and calibrates position-switch scans using `GBTFITSLoad.getps()`, which returns a `ScanBlock` object.  OffOn observations can be reduced the same way.

In [3]:
import os
import wget
import astropy.units as u
from dysh.fits.gbtfitsload import GBTFITSLoad

## First, we download the example SDFITS data, if necessary.


In [4]:
filename = "TGBT21A_501_11.raw.vegas.fits"
if not os.path.isfile(filename):
    url = f"http://www.gb.nrao.edu/dysh/example_data/onoff-L/data/{filename}"
    print(f"Downloading {filename}")
    wget.download(url,out=filename)
    print(f"\nRetrieved {filename}")
else:
    print(f"{filename} already downloaded")

TGBT21A_501_11.raw.vegas.fits already downloaded


In [5]:
sdfits = GBTFITSLoad(filename)
sdfits.info()

Filename: TGBT21A_501_11.raw.vegas.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      12   ()      
  1  SINGLE DISH    1 BinTableHDU    245   6040R x 74C   ['32A', '1D', '22A', '1D', '1D', '1D', '32768E', '16A', '6A', '8A', '1D', '1D', '1D', '4A', '1D', '4A', '1D', '1I', '32A', '32A', '1J', '32A', '16A', '1E', '8A', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '8A', '1D', '1D', '12A', '1I', '1I', '1D', '1D', '1I', '1A', '1I', '1I', '16A', '16A', '1J', '1J', '22A', '1D', '1D', '1I', '1A', '1D', '1E', '1D', '1D', '1D', '1D', '1D', '1A', '1A', '8A', '1E', '1E', '16A', '1I', '1I', '1I']   


In [9]:
print(sdfits.summary())

In [ ]:
psscan = sdfits.getps(152, ifnum=0, plnum=0)
psscan.calibrate()

In [ ]:
print(f"T_sys = {psscan[0].tsys.mean():.2f}")

In [ ]:
ta = psscan.timeaverage(weights='tsys')

In [ ]:
ta[0].plot()

In [ ]:
ta[0].plot(xaxis_unit="km/s",yaxis_unit="mK",ymin=-100,ymax=500,xmin=3000,xmax=4500)

In [ ]:
kms = u.km/u.s
ta[0].baseline(degree=2,exclude=[3600*kms,4100*kms],remove=True)

In [ ]:
ta[0].plot(ymin=-250)

In [ ]:
print(ta[0].baseline_model)

In [ ]:
ta[0].savefig("baselined_removed.png")